# PLSE ETL - Part 1: "13 years of data" to DB

In [ ]:
import pandas as pd # used only to convert xlsx to csv

In [ ]:
from functional import seq 

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession \
        .builder \
        .appName('plse') \
        .getOrCreate()

**While using this notebook as a development tool, if possible, AVOID redeclaration of `case_data`**

It takes some time to load the 3m+ rows x N cols with limited resources. Takes about 20-30 min with 16g RAM w/o pre dist load or clustering. This can be sorted when/if with containerize "etl related tools" in the future.

In [ ]:
#case_data = pd.ExcelFile('../tmp/PA2459713_Philadelphia_CaseData_Deliverable.xlsx')

In [ ]:
def extract(x):
    f = '../tmp/case_data_{}.csv'.format(x)
    #df = pd.read_excel(case_data, 'Case Data {}'.format(x))
    #df.to_csv(f)
    
    return spark.read.format('csv').option('header', 'true').load(f)

**While using this notebook as a development tool, if possible, AVOID redeclaration of `case_data_dfs`**

It does not take long, 7-9 min on average, but annoying so...good for thought

In [ ]:
case_data_dfs = [extract(i) for i in seq(1, 2, 3, 4)]

**Create "Wide" CaseData Spark Dataframe**

In [ ]:
df = spark.createDataFrame([],case_data_dfs[0].schema)

for _df in case_data_dfs:
    df = df.union(_df)

In [ ]:
df_count = df.count()

In [ ]:
assert df_count == 3668286, 'CaseData Dataframe Corrupt'

**In-memory lazy analysis**

Note: ATM in the code `df` is a spark dataframe. It contains ALL of the the case data.

In [ ]:
df.printSchema()

*Note: `df.OffenseTrackingNumber` is a string - see `df.printSchema()` stdout for more info*

In [ ]:
null_otn = df.filter(df.OffenseTrackingNumber != "").count() # rows in the in-memory dataframe without a OTN value

In [ ]:
print(df_count / null_otn, '%') # How many rows are have "no value" for OTN === 1.75% (of 3668286 rows)

*Note: if you uncomment then run the block below you'll see the first five row of the in-memory database*

In [ ]:
# print(df.show(5))

In [ ]:
df.toPandas().to_csv('../tmp/processed_case_data.csv')

**Load into PostgresSQL Database**

In [ ]:
# ...TDB, where will this data persist?

In [ ]:
df.show(5)